In [1]:
import pandas as pd
import numpy as np
import json

cols_needed = ['iyear', 'attacktype1_txt', 'success', 'nkill', 'nwound', 'property', 'propvalue']
df = pd.read_csv('terrorism_dataset.csv', encoding='ISO-8859-1', usecols=cols_needed, low_memory=False)

target_attacks_map = {
    'Bombing/Explosion': 'explosion',
    'Armed Assault': 'armed_assault',
    'Assassination': 'assassination',
    'Hostage Taking (Kidnapping)': 'hostage_taking',
    'Facility/Infrastructure Attack': 'infrastructure_attack'
}

target_attacks_list = list(target_attacks_map.keys())
df = df[df['attacktype1_txt'].isin(target_attacks_list)].copy()

cols_standard = ['success', 'nkill', 'nwound', 'attacktype1_txt', 'iyear']
df_clean = df.dropna(subset=cols_standard).copy()

df_clean['propvalue'] = pd.to_numeric(df_clean['propvalue'], errors='coerce')
df_clean.loc[df_clean['property'] != 1, 'propvalue'] = np.nan
df_clean = df_clean[(df_clean['iyear'] >= 1970) & (df_clean['iyear'] <= 2020)]

grouped = df_clean.groupby(['attacktype1_txt', 'iyear']).agg(
    success_rate=('success', 'mean'),
    avg_kills=('nkill', 'mean'),
    avg_wounded=('nwound', 'mean'),
    avg_damage=('propvalue', 'mean'),
    count=('iyear', 'count')
).reset_index()

output_data = {}
full_years = range(1970, 2021)

for real_name, simple_name in target_attacks_map.items():
    subset = grouped[grouped['attacktype1_txt'] == real_name]
    attack_data_list = []
    
    for year in full_years:
        record = subset[subset['iyear'] == year]
        entry = {"year": str(year)}
        
        if not record.empty:
            rec = record.iloc[0]
            entry["success_rate"] = float(rec['success_rate'])
            entry["avg_kills"] = float(rec['avg_kills'])
            entry["avg_wounded"] = float(rec['avg_wounded'])
            entry["count"] = int(rec['count'])
            entry["avg_damage"] = float(rec['avg_damage']) if not pd.isna(rec['avg_damage']) else 0.0
        else:
            entry["success_rate"] = 0.0
            entry["avg_kills"] = 0.0
            entry["avg_wounded"] = 0.0
            entry["avg_damage"] = 0.0
            entry["count"] = 0
            
        attack_data_list.append(entry)
    
    output_data[simple_name] = attack_data_list

with open('rose_chart_data.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2)